In [1]:

import numpy as np
import pandas as pd
import re
import os
import utils


In [ ]:
# main()

def read_tradelog():
    return


def filter_tradelog(df, tradelog_filters):
    for filt in tradelog_filters:
        func = globals()["utils.tlg_filter_" + filt]
        df = df[df.apply(func, axis=1)]
    return df


def multiple_group_tradelog(df, num_of_features, group_features, whole_group_features, calc_features_dict, feature_indicator_dict, output_dataframe):
    '''
    Function:
        Designate two lists of features: "group features" and "calc features". The "group features" are used for ".groupby()", and \
        the "calc features" for ".sum()". 
    
    Arguments:
        df:                                  dataframe:  input, i.e. the trade logs ready to be processed
        num_of_features:                     int:  the number of "group features"
        group_features:                      list:  a list of the names of "group features", indicating the current location of the recursion
        whole_group_features:                const list:  a whole list of the names of "group features"
        calc_features_dict:                  const dictionary:  (key, value) = (name of "calc features", a function to convert each row then "sum()")
        feature_indicator_dict:              const dictionary:  (key, value) = (name of "group features", a dictionary giving the range of this feature)
        output_dataframe:                    dataframe:  output
    
    '''
    if num_of_features == len(group_features):
        df["multiple_group_tradelog_indicator"] = True
    if num_of_features == 0:
        new_output_dataframe = df.copy()
        filter_new_output_columns = whole_group_features.copy()
        for calc_feature in calc_features_dict.keys():
            new_output_dataframe["tmp_calc_" + calc_feature] = new_output_dataframe.apply(calc_features_dict[calc_feature], axis=1)
            filter_new_output_columns.append("tmp_calc_" + calc_feature)
        new_output_dataframe = new_output_dataframe[filter_new_output_columns]
        new_output_dataframe = new_output_dataframe.groupby(whole_group_features).sum()
        new_output_dataframe.columns = calc_features_dict.keys()
        new_output_dataframe = new_output_dataframe.reset_index()
        output_dataframe = pd.concat(output_dataframe, new_output_dataframe)
    else:
        num_of_features -= 1
        current_feature = group_features.pop(0)
        current_feature_dict = feature_indicator_dict["current_feature"]
        for feature_value in current_feature_dict.keys():
            df_copy = df.copy()
            df_copy["multiple_group_tradelog_indicator"] = df_copy["multiple_group_tradelog_indicator"] & df_copy[feature_value].isin(current_feature_dict[feature_value])
            multiple_group_tradelog(df_copy[df_copy["multiple_group_tradelog_indicator"] == True], num_of_features, group_features.copy(), feature_indicator_dict)
    return


def group_tradelog(df, group_features, calc_features_dict, feature_indicator_dict):
    tradelog = df.copy()
    output = None
    multiple_group_tradelog(tradelog, len(group_features), group_features.copy(), group_features, calc_features_dict, feature_indicator_dict, output)
    return output


def add_features_to_tradelog(df, features):
        '''
    Function:
        Add a list of columns after grouping
    
    Arguments:
        df:                                  dataframe:  input, i.e. the trade logs ready to be processed
        features:                            list:  a list of names of features to be attached, and the conrresponding functions can be found /
                                                    in the folder: ???
        
    Return:
        output:                              dataframe:  output
    
    '''
    for feat in stats_features:
        func = globals(["utils.feature_" + feat])
        df[feat] = df.apply(func, axis=1)
    return df


In [ ]:
# import lib.tlg_filters as tlg_filters

def tlg_pipeline():
    # locate tradelogs
    input_dir = ""
    output_dir = ""
    
    # read tradelogs
    date = ["", ""]
    df = read_tradelog(input_dir, date)
    
    # filter tradelogs
    tradelog_filters = []
    df = filter_tradelog(df, tradelog_filters)
    
    # attach columns
    before_grouping_features = []
    df = add_features_to_tradelog(df, before_grouping_features)
    
    # group tradelogs
    group_features = []
    df = group_tradelog(df, group_features)
    
    # attach columns
    after_grouping_features = []
    df = add_features_to_tradelog(df, after_grouping_features)
    
    # end
    file_name = ""
    df.to_csv(file_name + ".csv")
    
    return


    